# Hands-on: Classification of Liver With or Without Tumor on CT Images
## 02 CNN Classification
Dataset reference: [Medical Segmentation Decathlon](http://medicaldecathlon.com/)

## 0\. Change to GPU
開啟此 notebook 後，請先至左上角「執行階段」內點選
「變更執行階段類型」，並將硬體加速器改為「GPU」

## 1\. Prepare the Enviornment

In [ ]:
import os
import numpy as np
import nibabel as nib
from google.colab import drive
from random import shuffle
from collections import Counter
from tqdm import tqdm_notebook as tqdm
from random import randint
from scipy.ndimage import zoom
from matplotlib import pyplot as plt

In [ ]:
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from sklearn.metrics import confusion_matrix, roc_curve, auc

## 2\. Download the Dataset

In [ ]:
drive.mount('/content/drive')

In [ ]:
os.chdir('/content/drive/MyDrive')

In [ ]:
# unzip the file
if not os.path.exists(r"./liver_classification") and os.path.exists(r"./liver_classification.zip"):
    !unzip liver_classification.zip

## 3\. Import and View the Data


### 3-1. Check the dataset
Let's check the amount of images we have.

In [ ]:
# Set the path
data_dir = './liver_classification'
image_dir = os.path.join(data_dir, 'crop_image')
label_dir = os.path.join(data_dir, 'crop_label')

In [ ]:
# Check how much data we have
print("Number of images:", len(os.listdir(image_dir)))
print("Number of labels:", len(os.listdir(label_dir)))

We have 130 pairs of CT images and labels.

### 3-2. Separate the images 分成訓練、驗證、測試
Then, we need to split the 130 data into training, validation, and testing sets. \
You can decide the amount of each set.

In [ ]:
# Separate data into training, validation, and testing set with ratio [0.6, 0.2, 0.2]
total_filelist = os.listdir(image_dir)    #將圖片資料夾中的圖片檔案名全部存到total_filelist
shuffle(total_filelist)     #將total_filelist隨機排序

############### change here #################
train_n = 78
valid_n = 26
test_n = 26
############### change here #################

train_list = total_filelist[:train_n]
valid_list = total_filelist[train_n : train_n+valid_n]
test_list = total_filelist[train_n + valid_n : train_n + valid_n + test_n]

print("{} data for training, list: {}".format(len(train_list), train_list))
print("{} data for validation, list: {}".format(len(valid_list), valid_list))
print("{} data for testing, list: {}".format(len(test_list), test_list))

### 3-3. Load and preprocess the images
在我們分離數據之後，我們必須加載數據並進行前處理，以便 


1.   調整數據格式以適應模型
2.   進行圖像處理以避免無用信息

下面是圖像預處理和數據加載的兩個基本函數


In [ ]:
# function of image resampling
def resample(image, label, output_shape=(224, 224)):
  # resample the image (make all images to the assigned size)
  resize_factor_x = output_shape[0] / np.shape(image)[0]
  resize_factor_y = output_shape[1] / np.shape(image)[1]
  image = zoom(image, (resize_factor_x, resize_factor_y), order=0, mode='nearest')
  label = zoom(label, (resize_factor_x, resize_factor_y), order=0, mode='nearest')

  return image, label

下面的函數執行數據加載和前處理。 \
前處理包括： 

*   **裁剪**：去除肝臟邊界框外的區域，您可以決定是否對圖像進行裁剪。
*   **重❤️採樣**：將圖像縮放到特定的形狀，我們模型的輸入形狀是(224, 224)（導入模型的時候可以找到），所以我們要把圖像重採樣成(224, 224)。
*   **加窗Windowin**：調整圖像的亮度和對比度，此過程僅對CT圖像有效，您可以針對不同的窗寬和窗位調整該值。
*   **正規化Normalization**：將每個像素上的值更改為 (0, 1)。
*   **掩蔽Masking**：將肝臟以外的區域改為0。您可以決定是否對圖像進行掩蔽。



In [ ]:
# function of data loading and preprocessing
def load_data(image_dir, label_dir, file_list, output_shape=(224, 224), channel=3, window_range=(-50, 200), perform_crop=True, perform_mask=True):
  X = []
  y = []

  #您可以決定是否對圖像進行裁剪。如果要剪裁，就除肝臟邊界框外的區域。
  if perform_crop:
    threshold_tumor = 0.001
    threshold_liver = 0.04
  else:
    threshold_tumor = 0.002
    threshold_liver = 0.08

  #我要利用檔名，將每張圖片讀取出來
  for filename in tqdm(file_list):      #tqdm：進度條
    ori_image = nib.load(os.path.join(image_dir, filename)).get_fdata()
    ori_label = nib.load(os.path.join(label_dir, filename)).get_fdata()

    #剪裁（？？）
    # Preprocessing - cropping
    if perform_crop:
      x_min = np.min(np.where(ori_label != 0)[0])
      x_max = np.max(np.where(ori_label != 0)[0])
      y_min = np.min(np.where(ori_label != 0)[1])
      y_max = np.max(np.where(ori_label != 0)[1])
      ori_image = ori_image[x_min:x_max, y_min:y_max, :]
      ori_label = ori_label[x_min:x_max, y_min:y_max, :]


    for idx in range(ori_image.shape[2]):
      if not np.max(ori_label[:, :, idx]) == 0:
        # Preprocessing - resampling
        image_2d, label_2d = resample(ori_image[:, :, idx], ori_label[:, :, idx], output_shape=output_shape)
        # Preprocessing - windowing
        image_2d = np.clip(image_2d, window_range[0], window_range[1])
        # Preprocessing - normalization
        image_2d = (image_2d - np.min(image_2d))/(np.max(image_2d) - np.min(image_2d))
        # Preprocessing - masking
        if perform_mask:
          image_2d[np.where(label_2d == 0)] = 0

        if np.max(label_2d) == 2 and np.count_nonzero(label_2d == 1) > output_shape[0]**2*threshold_tumor:
          X.append(image_2d)
          y.append(1)
        elif np.max(label_2d) == 1 and np.count_nonzero(label_2d == 1) > output_shape[0]**2*threshold_liver:
          X.append(image_2d)
          y.append(0)

  X = np.array(X)
  stacked_X = np.stack((X,)*channel, axis=-1)
  print("We got {} 2D images with {}".format(len(y), Counter(y)))
  y = np.array(y)
  print("Done loading data with X shape = {}, y shape = {}".format(stacked_X.shape, y.shape))

  return stacked_X, y

您可以在此處調整一些用於前處理的參數。



In [ ]:
# Load data and perform image processing
############### change here #################
output_shape = (224, 224)
channel = 3
window_range = (-50, 200)
perform_crop = True
perform_mask = True
############### change here #################

train_X, train_y = load_data(
    image_dir, label_dir, train_list, 
    output_shape=output_shape, 
    channel=channel, 
    window_range=window_range,
    perform_crop=perform_crop, 
    perform_mask=perform_mask
)

valid_X, valid_y = load_data(
    image_dir, label_dir, valid_list, 
    output_shape=output_shape, 
    channel=channel, 
    window_range=window_range,
    perform_crop=perform_crop, 
    perform_mask=perform_mask
)

### 3-4. Visualize the data
After loading the data, let's have a look at what `train_X`, `train_y`, `valid_X`, `valid_y` we got.

In [ ]:
# function of plotting data randomly
def plot_image(X, y, row, col):
  ax = []
  fig = plt.figure(figsize=(3*col, 3*row))


  for i in range(row*col):
    value = randint(0, X.shape[0]-1)
    image_plot = X[value, :, :, 0]
    ax.append(fig.add_subplot(row, col, i+1))
    ax[i].set_title("Label: {}".format(y[value]))
    plt.axis('off')
    plt.imshow(image_plot, cmap='gray')
  plt.show()

We can randomly pick some examples from training or validation sets to see the images.\
You can change how many images to show at each row or column run the code repeatly.

In [ ]:
############### change here #################
row = 1
col = 5
############### change here #################

plot_image(train_X, train_y, row, col)

### 3-5. Fit the data into generator with augmentation
We can perform augmentation to increase the variability of the training data. \
This code shows the original parameters (aka without any augmentation). \
You can try different number. For example, setting `rotation_range`=10, `width_shift_range`=0.1, `height_shift_range`=0.1, `shear_range`=0.1, `zoom_range`=0.1) \
There's no "best" setting for deep learning. All you can do is running several experiment to find the pattern. \
You can check more detail at [ImageDataGenerator class](https://keras.io/api/preprocessing/image/#imagedatagenerator-class)

In [ ]:
# Set the augmentation parameters and fit the training data
############### change here #################
datagen = ImageDataGenerator(
    rotation_range=0,
    width_shift_range=0.0,
    height_shift_range=0.0,
    shear_range=0.0,
    zoom_range=0.0,
    fill_mode="constant",
    cval=0
)
############### change here #################
datagen.fit(train_X)

## 4\. Build the Model
We finished preparing the data! \
It's time to build a CNN model for classification if the CT iamge contains a liver tumor or not.\
Here we use MobileNet provided by Keras Applications because this may be the fastest model. \
You can also visit [Keras Applications](https://keras.io/api/applications/) to try different model.

### 4-1. Load the model with pretrained weights

In [ ]:
from tensorflow.keras.applications import MobileNet

In [ ]:
# Load the model and add layers for binary classification output
base_model = MobileNet(
    input_shape=(224, 224, 3),
    include_top=False,
    weights="imagenet",
    input_tensor=None,
    pooling=None,
    classes=1000
)

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)
model = Model(inputs=base_model.input, outputs=predictions)

In [ ]:
# Compile the model
model.compile(
      optimizer=keras.optimizers.Adam(1e-4),
      loss="binary_crossentropy",
      metrics=["accuracy"],
)

In [ ]:
# [optional] Plot the model (method 1)
keras.utils.plot_model(model, show_shapes=True)

In [ ]:
# [optional] Plot the model (method 2)
model.summary()

### 4-2. Start training model
We start training the model here. \
Please notice that this might take a while. You can also adjust the number of epochs.


In [ ]:
# Set the epochs and batch size, then train the model
############### change here #################
epochs = 5
batch_size = 32
############### change here #################

history = model.fit(
    datagen.flow(train_X, train_y, batch_size=batch_size),
    steps_per_epoch=len(train_X)/batch_size,
    epochs=epochs,
    validation_data=(valid_X, valid_y)
)

When we finished training, we can take a look at the changes of  loss and accuracy on training and validation data during training.

In [ ]:
# Plot loss and accuracy
fig = plt.figure(figsize=(15, 5))
ax1 = fig.add_subplot(1, 2, 1)
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Train History')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['train', 'validation'], loc='upper left')

ax2 = fig.add_subplot(1, 2, 2)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Train History')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['train', 'validation'], loc='upper left')

## 5\. Evaluate the Test Data
We've done training the model. \
Now it's time to see how the model perform on testing data.\
Firstly, let's load the testing data using the same method when we load the training and validation data.

In [ ]:
# Load test data
test_X, test_y = load_data(
    image_dir, label_dir, test_list, 
    output_shape=output_shape, 
    channel=channel, 
    window_range=window_range,
    perform_crop=perform_crop, 
    perform_mask=perform_mask
)

Here we perform the trained mdoel on the test data.

In [ ]:
# Predict the test data
predict_y = model.predict(test_X)

Then, we can see the classification results.

In [ ]:
# Plot the ROC curve of the test results
plt.figure()
plt.plot([0, 1], [0, 1], 'k--')

fpr, tpr, _ = roc_curve(test_y, predict_y)
roc_auc = auc(fpr, tpr)
plt.plot(fpr, tpr, label='AUC = {}'.format(roc_auc))

plt.legend(loc='lower right')
plt.xlim([0, 1])
plt.ylim([0, 1.05])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')

We now turn the result into binary result, and make the comparison on the groundtruths by the confusion matrix.

In [ ]:
# Print the confusion matrix of the binray classification
predict_y[predict_y >= 0.5] = 1
predict_y[predict_y < 0.5] = 0

print(confusion_matrix(test_y, predict_y, labels=[1, 0]))

Then, we can check the sensitivity and specificity.

In [ ]:
# Calculate the sensitivity and specificity
TP = confusion_matrix(test_y, predict_y, labels=[1, 0])[0, 0]
FP = confusion_matrix(test_y, predict_y, labels=[1, 0])[1, 0]
FN = confusion_matrix(test_y, predict_y, labels=[1, 0])[0, 1]
TN = confusion_matrix(test_y, predict_y, labels=[1, 0])[1, 1]
print("True positive: {}".format(TP))
print("False positive: {}".format(FP))
print("False negative: {}".format(FN))
print("True negative: {}".format(TN))


############### your code here #################
sensitivity =
specificity =
############### your code here #################

print("Sensitivity: {}".format(sensitivity))
print("Specificity: {}".format(specificity))